In [ ]:
import pandas as pd
import numpy as np
import math
from sklearn import metrics
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import model_selection
from sklearn.model_selection import GridSearchCV
import xgboost as xgb

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/1. DNA/[4]table/Embedding_train_data.csv')
test_df = pd.read_csv('/content/drive/MyDrive/1. DNA/[4]table/Embedding_test_data.csv')

In [ ]:
sub = pd.read_csv('/content/drive/MyDrive/1. DNA/[4]table/sample_submission.csv')

In [ ]:
train_df.head()

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴,...,석식41,석식42,석식43,석식44,석식45,석식46,석식47,석식48,석식49,석식50
0,2016-02-01,월,2601,50,150,238,0.0,"['모닝롤', '찐빵', '우유', '두유', '주스', '계란후라이', '호두죽'...","['쌀밥', '잡곡밥', '(쌀,현미흑미:국내산)', '오징어찌개', '쇠불고기',...","['쌀밥', '잡곡밥', '(쌀,현미흑미:국내산)', '육개장', '자반고등어구이'...",...,-0.677969,0.026655,0.521240,0.106772,-0.151797,0.285805,-0.916271,-0.327705,-0.489872,-0.114326
1,2016-02-02,화,2601,50,173,319,0.0,"['모닝롤', '단호박샌드', '우유', '두유', '주스', '계란후라이', '팥...","['쌀밥', '잡곡밥', '(쌀,현미흑미:국내산)', '김치찌개', '가자미튀김',...","['콩나물밥*양념장', '(쌀,현미흑미:국내산)', '어묵국', '유산슬', '(쇠...",...,0.218089,-0.672441,0.787501,0.266674,-0.693809,0.412709,-0.846675,-0.540325,-0.346797,0.396869
2,2016-02-03,수,2601,56,180,111,0.0,"['모닝롤', '베이글', '우유', '두유', '주스', '계란후라이', '표고버...","['카레덮밥', '(쌀,현미흑미:국내산)', '팽이장국', '치킨핑거', '(닭고기...","['쌀밥', '잡곡밥', '(쌀,현미흑미:국내산)', '청국장찌개', '황태양념구이...",...,-0.149827,0.259681,0.897299,0.419382,0.033570,0.816938,-0.607790,-0.501021,0.063332,0.382068
3,2016-02-04,목,2601,104,220,355,0.0,"['모닝롤', '토마토샌드', '우유', '두유', '주스', '계란후라이', '닭...","['쌀밥', '잡곡밥', '(쌀,현미흑미:국내산)', '쇠고기무국', '주꾸미볶음'...","['미니김밥*겨자장', '(쌀,현미흑미:국내산)', '우동', '멕시칸샐러드', '...",...,-0.257097,-0.550777,1.180077,0.568659,-0.483393,1.447895,-0.850024,-0.630203,-0.074369,0.335166
4,2016-02-05,금,2601,278,181,34,0.0,"['모닝롤', '와플', '우유', '두유', '주스', '계란후라이', '쇠고기죽...","['쌀밥', '잡곡밥', '(쌀,현미흑미:국내산)', '떡국', '돈육씨앗강정', ...","['쌀밥', '잡곡밥', '(쌀,현미흑미:국내산)', '차돌박이찌개', '(쇠고기:...",...,-0.289660,0.011186,1.035358,0.692184,-0.199249,0.299185,-0.974069,-0.377276,-0.117614,-0.063785


In [ ]:
train_df = train_df.drop(['조식메뉴','중식메뉴','석식메뉴'], axis = 1)
train_df.head()

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,중식계,석식계,조식1,...,석식41,석식42,석식43,석식44,석식45,석식46,석식47,석식48,석식49,석식50
0,2016-02-01,월,2601,50,150,238,0.0,1039.0,331.0,0.345063,...,-0.677969,0.026655,0.521240,0.106772,-0.151797,0.285805,-0.916271,-0.327705,-0.489872,-0.114326
1,2016-02-02,화,2601,50,173,319,0.0,867.0,560.0,0.537751,...,0.218089,-0.672441,0.787501,0.266674,-0.693809,0.412709,-0.846675,-0.540325,-0.346797,0.396869
2,2016-02-03,수,2601,56,180,111,0.0,1017.0,573.0,0.430975,...,-0.149827,0.259681,0.897299,0.419382,0.033570,0.816938,-0.607790,-0.501021,0.063332,0.382068
3,2016-02-04,목,2601,104,220,355,0.0,978.0,525.0,0.695163,...,-0.257097,-0.550777,1.180077,0.568659,-0.483393,1.447895,-0.850024,-0.630203,-0.074369,0.335166
4,2016-02-05,금,2601,278,181,34,0.0,925.0,330.0,0.519855,...,-0.289660,0.011186,1.035358,0.692184,-0.199249,0.299185,-0.974069,-0.377276,-0.117614,-0.063785


In [ ]:
test_df = test_df.drop(['조식메뉴','중식메뉴','석식메뉴'], axis = 1)
test_df.head()

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식1,조식2,조식3,...,석식41,석식42,석식43,석식44,석식45,석식46,석식47,석식48,석식49,석식50
0,2021-01-27,수,2983,88,182,5,358.0,0.318706,1.173837,0.228873,...,0.855815,-0.676641,0.008660,0.049113,0.279932,-0.255711,-0.784561,-0.287553,-0.123809,-0.417225
1,2021-01-28,목,2983,104,212,409,348.0,0.273322,0.871655,0.077482,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,2021-01-29,금,2983,270,249,0,294.0,0.068761,1.216680,0.287047,...,1.274020,-0.963867,0.068798,0.263769,0.199401,-0.475091,-1.022334,-0.346184,0.053542,-0.519112
3,2021-02-01,월,2924,108,154,538,322.0,0.051651,1.120480,0.057369,...,1.215914,-0.308417,0.200664,-0.110430,0.222746,-0.128564,-0.394535,-0.758083,-0.181848,-0.035184
4,2021-02-02,화,2924,62,186,455,314.0,0.405712,1.120068,0.380671,...,2.021988,-0.430297,-0.483459,-0.619689,0.896096,-0.793497,-0.771993,-0.141780,0.286809,-0.650598


In [ ]:
train_df = train_df.rename(columns = {'일자':'date','요일':'week'})
test_df = test_df.rename(columns = {'일자':'date','요일':'week'})

In [ ]:
train_df.head(2)

,date,week,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,중식계,석식계,조식1,...,석식41,석식42,석식43,석식44,석식45,석식46,석식47,석식48,석식49,석식50
0,2016-02-01,월,2601,50,150,238,0.0,1039.0,331.0,0.345063,...,-0.677969,0.026655,0.521240,0.106772,-0.151797,0.285805,-0.916271,-0.327705,-0.489872,-0.114326
1,2016-02-02,화,2601,50,173,319,0.0,867.0,560.0,0.537751,...,0.218089,-0.672441,0.787501,0.266674,-0.693809,0.412709,-0.846675,-0.540325,-0.346797,0.396869


In [ ]:
from pandas.core.common import temp_setattr
train_df['year'] = pd.to_datetime(train_df['date']).dt.year.astype(int)
train_df['month'] = pd.to_datetime(train_df['date']).dt.month.astype(int)
train_df['day'] = pd.to_datetime(train_df['date']).dt.day.astype(int)

test_df['year'] = pd.to_datetime(test_df['date']).dt.year.astype(int)
test_df['month'] = pd.to_datetime(test_df['date']).dt.month.astype(int)
test_df['day'] = pd.to_datetime(test_df['date']).dt.day.astype(int)

In [ ]:
train_df.head(2)

,date,week,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,중식계,석식계,조식1,...,석식44,석식45,석식46,석식47,석식48,석식49,석식50,year,month,day
0,2016-02-01,월,2601,50,150,238,0.0,1039.0,331.0,0.345063,...,0.106772,-0.151797,0.285805,-0.916271,-0.327705,-0.489872,-0.114326,2016,2,1
1,2016-02-02,화,2601,50,173,319,0.0,867.0,560.0,0.537751,...,0.266674,-0.693809,0.412709,-0.846675,-0.540325,-0.346797,0.396869,2016,2,2


In [ ]:
test_df.head(2)

,date,week,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식1,조식2,조식3,...,석식44,석식45,석식46,석식47,석식48,석식49,석식50,year,month,day
0,2021-01-27,수,2983,88,182,5,358.0,0.318706,1.173837,0.228873,...,0.049113,0.279932,-0.255711,-0.784561,-0.287553,-0.123809,-0.417225,2021,1,27
1,2021-01-28,목,2983,104,212,409,348.0,0.273322,0.871655,0.077482,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2021,1,28


In [ ]:
# 기존 날짜 피처 삭제
train_df.drop(columns=['date'], inplace=True)
test_df.drop(columns=['date'], inplace=True)

train_df.shape, test_df.shape

((1205, 161), (50, 159))

In [ ]:
train_df.replace({"week": ["월","화","수","목","금"] }, {"week": [0,1,2,3,4] }, inplace = True)
test_df.replace({"week": ["월","화","수","목","금"] }, {"week": [0,1,2,3,4] }, inplace = True)
train_df.head(3)

,week,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,중식계,석식계,조식1,조식2,...,석식44,석식45,석식46,석식47,석식48,석식49,석식50,year,month,day
0,0,2601,50,150,238,0.0,1039.0,331.0,0.345063,0.841029,...,0.106772,-0.151797,0.285805,-0.916271,-0.327705,-0.489872,-0.114326,2016,2,1
1,1,2601,50,173,319,0.0,867.0,560.0,0.537751,0.772565,...,0.266674,-0.693809,0.412709,-0.846675,-0.540325,-0.346797,0.396869,2016,2,2
2,2,2601,56,180,111,0.0,1017.0,573.0,0.430975,0.564405,...,0.419382,0.033570,0.816938,-0.607790,-0.501021,0.063332,0.382068,2016,2,3


In [ ]:
test_df.replace({"week": ["월","화","수","목","금"] }, {"week": [0,1,2,3,4] }, inplace = True)
test_df.replace({"week": ["월","화","수","목","금"] }, {"week": [0,1,2,3,4] }, inplace = True)
test_df.head(3)

,week,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식1,조식2,조식3,조식4,...,석식44,석식45,석식46,석식47,석식48,석식49,석식50,year,month,day
0,2,2983,88,182,5,358.0,0.318706,1.173837,0.228873,-0.016794,...,0.049113,0.279932,-0.255711,-0.784561,-0.287553,-0.123809,-0.417225,2021,1,27
1,3,2983,104,212,409,348.0,0.273322,0.871655,0.077482,0.143496,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2021,1,28
2,4,2983,270,249,0,294.0,0.068761,1.216680,0.287047,0.108864,...,0.263769,0.199401,-0.475091,-1.022334,-0.346184,0.053542,-0.519112,2021,1,29


In [ ]:
#날짜등이 주기성이 있다고 판단했으면 인코딩을 ex ) 카테고리로 하고 캣부스트 , 조류같은 걸 예상할 때는 다른 값으로 변경해줘야함 여기선 상관없다 ! 

In [ ]:
train_df['week'] = train_df['week'].astype('category')

In [ ]:
test_df['week'] = train_df['week'].astype('category')

In [ ]:
train_df.isna().sum()

week              0
본사정원수             0
본사휴가자수            0
본사출장자수            0
본사시간외근무명령서승인건수    0
                 ..
석식49              0
석식50              0
year              0
month             0
day               0
Length: 161, dtype: int64

In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler 

#정규화 변환
mm = MinMaxScaler()

In [ ]:
train_df2 = train_df.drop(['중식계','석식계'], axis = 1)

In [ ]:
train_df2

,week,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식1,조식2,조식3,조식4,...,석식44,석식45,석식46,석식47,석식48,석식49,석식50,year,month,day
0,0,2601,50,150,238,0.0,0.345063,0.841029,-0.244781,-0.099740,...,0.106772,-0.151797,0.285805,-0.916271,-0.327705,-0.489872,-0.114326,2016,2,1
1,1,2601,50,173,319,0.0,0.537751,0.772565,-0.227728,-0.403583,...,0.266674,-0.693809,0.412709,-0.846675,-0.540325,-0.346797,0.396869,2016,2,2
2,2,2601,56,180,111,0.0,0.430975,0.564405,0.014996,-0.049187,...,0.419382,0.033570,0.816938,-0.607790,-0.501021,0.063332,0.382068,2016,2,3
3,3,2601,104,220,355,0.0,0.695163,0.915111,-0.464399,-0.160095,...,0.568659,-0.483393,1.447895,-0.850024,-0.630203,-0.074369,0.335166,2016,2,4
4,4,2601,278,181,34,0.0,0.519855,0.525534,-0.039259,-0.389989,...,0.692184,-0.199249,0.299185,-0.974069,-0.377276,-0.117614,-0.063785,2016,2,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1200,2,2983,75,198,4,391.0,0.205926,1.241737,0.267033,0.033066,...,-0.263557,0.738902,0.129731,0.395461,-0.805688,-0.497606,0.216988,2021,1,20
1201,3,2983,92,231,462,351.0,-0.045411,1.020490,0.032205,-0.095292,...,0.122566,0.351240,-0.328923,-0.620603,-0.696990,0.313707,0.084509,2021,1,21
1202,4,2983,255,248,1,303.0,0.246507,1.052156,0.344443,0.081379,...,0.036668,0.654255,-0.277724,-0.561344,-0.290120,-0.088212,-0.819337,2021,1,22
1203,0,2983,107,153,616,327.0,0.092769,1.107356,0.117192,0.020893,...,-0.062785,-0.246567,-0.333712,0.029828,-0.845187,-0.185106,-0.561755,2021,1,25


In [ ]:
dd = train_df2.iloc[:, 1:-3]

In [ ]:
dt = test_df.iloc[:,1:-3]

In [ ]:
# train 스케일링
scale_cols = dd.columns
df_scaled = mm.fit_transform(train_df[scale_cols])
df_scaled = pd.DataFrame(df_scaled)
df_scaled.columns = scale_cols

df_scaled

,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식1,조식2,조식3,조식4,조식5,...,석식41,석식42,석식43,석식44,석식45,석식46,석식47,석식48,석식49,석식50
0,0.000000,0.022481,0.323442,0.227969,0.000000,0.509308,0.622950,0.541768,0.400148,0.703082,...,0.164914,0.365596,0.545378,0.310087,0.316197,0.597742,0.454880,0.480892,0.226945,0.482791
1,0.000000,0.022481,0.391691,0.305556,0.000000,0.666035,0.572860,0.550429,0.055091,0.688134,...,0.350518,0.146995,0.642989,0.366200,0.167465,0.630739,0.480192,0.414629,0.293238,0.665451
2,0.000000,0.027477,0.412463,0.106322,0.000000,0.579186,0.420564,0.673700,0.457558,0.722243,...,0.274310,0.438461,0.683240,0.419788,0.367064,0.735843,0.567076,0.426878,0.483270,0.660162
3,0.000000,0.067444,0.531157,0.340038,0.000000,0.794069,0.677150,0.430232,0.331606,0.711087,...,0.252091,0.185038,0.786906,0.472172,0.225205,0.899900,0.478974,0.386618,0.419467,0.643403
4,0.000000,0.212323,0.415430,0.032567,0.000000,0.651479,0.392125,0.646145,0.070529,0.745093,...,0.245346,0.360759,0.733852,0.515519,0.303176,0.601222,0.433859,0.465443,0.399429,0.500850
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1200,0.542614,0.043297,0.465875,0.003831,0.733583,0.396137,0.916118,0.801701,0.550967,0.263018,...,0.337892,0.342153,0.543685,0.180132,0.560612,0.557161,0.931962,0.331928,0.223361,0.601176
1201,0.542614,0.057452,0.563798,0.442529,0.658537,0.191706,0.754248,0.682439,0.405199,0.346025,...,0.446457,0.251244,0.395963,0.315629,0.454235,0.437906,0.562416,0.365804,0.599280,0.553839
1202,0.542614,0.193172,0.614243,0.000958,0.568480,0.429144,0.777416,0.841015,0.605833,0.316439,...,0.522061,0.154516,0.399177,0.285486,0.537384,0.451218,0.583968,0.492605,0.413052,0.230876
1203,0.542614,0.069942,0.332344,0.590038,0.613508,0.304097,0.817802,0.725602,0.537143,0.360469,...,0.531137,0.374476,0.431320,0.250586,0.290192,0.436660,0.798980,0.319618,0.368157,0.322915


In [ ]:
# test 스케일링
scale_cols = dt.columns
dt_scaled = mm.transform(test_df[scale_cols])
dt_scaled = pd.DataFrame(dt_scaled)
dt_scaled.columns = scale_cols

dt_scaled

,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식1,조식2,조식3,조식4,조식5,...,석식41,석식42,석식43,석식44,석식45,석식46,석식47,석식48,석식49,석식50
0,0.542614,0.054122,0.418398,0.004789,0.671670,0.487869,0.866441,0.782321,0.494345,0.217995,...,0.482612,0.145681,0.357469,0.289854,0.434667,0.456942,0.502784,0.493405,0.396559,0.374559
1,0.542614,0.067444,0.507418,0.391762,0.652908,0.450955,0.645356,0.705434,0.676377,0.460899,...,0.305344,0.357261,0.354294,0.272619,0.357852,0.523430,0.788132,0.583021,0.453925,0.523642
2,0.542614,0.205662,0.617211,0.000000,0.551595,0.284570,0.897786,0.811865,0.637047,0.322710,...,0.569236,0.055868,0.379515,0.365180,0.412569,0.399900,0.416305,0.475133,0.478734,0.338152
3,0.458807,0.070774,0.335312,0.515326,0.604128,0.270653,0.827404,0.695220,0.594668,0.314394,...,0.557200,0.260822,0.427856,0.233867,0.418975,0.490002,0.644637,0.346764,0.369667,0.511070
4,0.458807,0.032473,0.430267,0.435824,0.589118,0.558638,0.827102,0.859413,0.867249,0.240834,...,0.724165,0.222711,0.177060,0.055158,0.603747,0.317111,0.507354,0.538836,0.586817,0.291170
5,0.458807,0.029975,0.468843,0.004789,0.536585,0.373675,0.980996,0.746250,0.528384,0.327268,...,0.305344,0.357261,0.354294,0.272619,0.357852,0.523430,0.788132,0.583021,0.453925,0.523642
6,0.458807,0.031640,0.504451,0.455939,0.540338,0.292359,0.663593,0.864073,0.805464,0.484263,...,0.569236,0.055868,0.379515,0.365180,0.412569,0.399900,0.416305,0.475133,0.478734,0.338152
7,0.458807,0.121565,0.626113,0.000000,0.480300,0.278093,0.911101,0.964048,0.613334,0.193145,...,0.138472,0.063668,0.759030,0.711394,0.000000,0.799800,0.703640,0.260905,0.259111,0.569656
8,0.458807,0.054122,0.394659,0.660920,0.617261,0.301714,0.674441,0.864622,0.497101,0.410160,...,0.541012,0.108659,0.326711,0.289500,0.433144,0.424059,0.561730,0.450340,0.507680,0.355378
9,0.458807,0.059117,0.421365,0.519157,0.617261,0.329916,0.837770,0.785933,0.351278,0.201361,...,0.697744,0.229935,0.243168,-0.118688,0.531403,0.232778,0.607676,0.465375,0.752809,0.321469


In [ ]:
scale_cols

Index(['본사정원수', '본사휴가자수', '본사출장자수', '본사시간외근무명령서승인건수', '현본사소속재택근무자수', '조식1',
       '조식2', '조식3', '조식4', '조식5',
       ...
       '석식41', '석식42', '석식43', '석식44', '석식45', '석식46', '석식47', '석식48', '석식49',
       '석식50'],
      dtype='object', length=155)

In [ ]:
# 중석식계스케일링
ff_scaled = mm.fit_transform(train_df[['중식계','석식계']])
ff_scaled = pd.DataFrame(ff_scaled, columns = ['중식계','석식계'])


ff_scaled

,중식계,석식계
0,0.638865,0.365746
1,0.490972,0.618785
2,0.619948,0.633149
3,0.586414,0.580110
4,0.540843,0.364641
...,...,...
1200,0.685297,0.465193
1201,0.460877,0.390055
1202,0.243336,0.239779
1203,0.730009,0.554696


In [ ]:
train_df[['중식계','석식계']]

,중식계,석식계
0,1039.0,331.0
1,867.0,560.0
2,1017.0,573.0
3,978.0,525.0
4,925.0,330.0
...,...,...
1200,1093.0,421.0
1201,832.0,353.0
1202,579.0,217.0
1203,1145.0,502.0


In [ ]:
tttt = train_df[['week','year','month','day']]

In [ ]:
train_df = pd.concat([tttt, ff_scaled,df_scaled], axis = 1)

In [ ]:
train_df

,week,year,month,day,중식계,석식계,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,...,석식41,석식42,석식43,석식44,석식45,석식46,석식47,석식48,석식49,석식50
0,0,2016,2,1,0.638865,0.365746,0.000000,0.022481,0.323442,0.227969,...,0.164914,0.365596,0.545378,0.310087,0.316197,0.597742,0.454880,0.480892,0.226945,0.482791
1,1,2016,2,2,0.490972,0.618785,0.000000,0.022481,0.391691,0.305556,...,0.350518,0.146995,0.642989,0.366200,0.167465,0.630739,0.480192,0.414629,0.293238,0.665451
2,2,2016,2,3,0.619948,0.633149,0.000000,0.027477,0.412463,0.106322,...,0.274310,0.438461,0.683240,0.419788,0.367064,0.735843,0.567076,0.426878,0.483270,0.660162
3,3,2016,2,4,0.586414,0.580110,0.000000,0.067444,0.531157,0.340038,...,0.252091,0.185038,0.786906,0.472172,0.225205,0.899900,0.478974,0.386618,0.419467,0.643403
4,4,2016,2,5,0.540843,0.364641,0.000000,0.212323,0.415430,0.032567,...,0.245346,0.360759,0.733852,0.515519,0.303176,0.601222,0.433859,0.465443,0.399429,0.500850
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1200,2,2021,1,20,0.685297,0.465193,0.542614,0.043297,0.465875,0.003831,...,0.337892,0.342153,0.543685,0.180132,0.560612,0.557161,0.931962,0.331928,0.223361,0.601176
1201,3,2021,1,21,0.460877,0.390055,0.542614,0.057452,0.563798,0.442529,...,0.446457,0.251244,0.395963,0.315629,0.454235,0.437906,0.562416,0.365804,0.599280,0.553839
1202,4,2021,1,22,0.243336,0.239779,0.542614,0.193172,0.614243,0.000958,...,0.522061,0.154516,0.399177,0.285486,0.537384,0.451218,0.583968,0.492605,0.413052,0.230876
1203,0,2021,1,25,0.730009,0.554696,0.542614,0.069942,0.332344,0.590038,...,0.531137,0.374476,0.431320,0.250586,0.290192,0.436660,0.798980,0.319618,0.368157,0.322915


In [ ]:
train_df.columns

Index(['week', 'year', 'month', 'day', '본사정원수', '본사휴가자수', '본사출장자수',
       '본사시간외근무명령서승인건수', '현본사소속재택근무자수', '중식계',
       ...
       '석식40', '석식41', '석식42', '석식43', '석식44', '석식45', '석식46', '석식47', '석식48',
       '석식49'],
      dtype='object', length=160)

In [ ]:
ttss = test_df[['week','year','month','day']]

In [ ]:
test_df = pd.concat([ttss, dt_scaled], axis = 1)

In [ ]:
test_df

,week,year,month,day,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식1,...,석식41,석식42,석식43,석식44,석식45,석식46,석식47,석식48,석식49,석식50
0,0,2021,1,27,0.542614,0.054122,0.418398,0.004789,0.671670,0.487869,...,0.482612,0.145681,0.357469,0.289854,0.434667,0.456942,0.502784,0.493405,0.396559,0.374559
1,1,2021,1,28,0.542614,0.067444,0.507418,0.391762,0.652908,0.450955,...,0.305344,0.357261,0.354294,0.272619,0.357852,0.523430,0.788132,0.583021,0.453925,0.523642
2,2,2021,1,29,0.542614,0.205662,0.617211,0.000000,0.551595,0.284570,...,0.569236,0.055868,0.379515,0.365180,0.412569,0.399900,0.416305,0.475133,0.478734,0.338152
3,3,2021,2,1,0.458807,0.070774,0.335312,0.515326,0.604128,0.270653,...,0.557200,0.260822,0.427856,0.233867,0.418975,0.490002,0.644637,0.346764,0.369667,0.511070
4,4,2021,2,2,0.458807,0.032473,0.430267,0.435824,0.589118,0.558638,...,0.724165,0.222711,0.177060,0.055158,0.603747,0.317111,0.507354,0.538836,0.586817,0.291170
5,3,2021,2,3,0.458807,0.029975,0.468843,0.004789,0.536585,0.373675,...,0.305344,0.357261,0.354294,0.272619,0.357852,0.523430,0.788132,0.583021,0.453925,0.523642
6,4,2021,2,4,0.458807,0.031640,0.504451,0.455939,0.540338,0.292359,...,0.569236,0.055868,0.379515,0.365180,0.412569,0.399900,0.416305,0.475133,0.478734,0.338152
7,0,2021,2,5,0.458807,0.121565,0.626113,0.000000,0.480300,0.278093,...,0.138472,0.063668,0.759030,0.711394,0.000000,0.799800,0.703640,0.260905,0.259111,0.569656
8,1,2021,2,8,0.458807,0.054122,0.394659,0.660920,0.617261,0.301714,...,0.541012,0.108659,0.326711,0.289500,0.433144,0.424059,0.561730,0.450340,0.507680,0.355378
9,2,2021,2,9,0.458807,0.059117,0.421365,0.519157,0.617261,0.329916,...,0.697744,0.229935,0.243168,-0.118688,0.531403,0.232778,0.607676,0.465375,0.752809,0.321469


###**LGBM** 

In [ ]:
from sklearn.model_selection import GridSearchCV , cross_val_score
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error

In [ ]:
def get_val_score(model):
    val_pred = model.predict_proba(X_valid)
    return  mean_absolute_error(y_valid,val_pred[:,1])

In [ ]:
#
target_col1 = '중식계'
target_col2 = '석식계'
train_cols = [col for col in train_df.columns if col not in [target_col1 , target_col2]]

In [ ]:
#'중식계'

X_train, X_valid, y_train, y_valid = train_test_split(train_df[train_cols], train_df[target_col1], test_size=0.2, shuffle=True, random_state=34)

lgb_params = {
     'max_depth' : [3,5,8] , 
     'n_estimators': [1000,1200,1500], 
     'learning_rate':[0.01,0.1], 
     'gamma': [0.5], 
     'random_state':[99] }

model = lgb.LGBMRegressor(eval_set=[(X_valid, y_valid)],eval_metric = 'mae')


grid_lgb = GridSearchCV(model, param_grid=lgb_params, cv=3)  # 최소 5 이상 사용 권장
grid_lgb.fit(X_train, y_train)
print(grid_lgb.best_params_)

#Get score
print("="*20)
#print("LGBM Validation MAE : ", get_val_score(grid_lgb))
print("Model1 Score: ",grid_lgb.score(X_valid,y_valid))
print("MAE:",mean_absolute_error(y_valid, grid_lgb.predict(X_valid)))

{'gamma': 0.5, 'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 1500, 'random_state': 99}
Model1 Score:  0.7938393027098887
MAE: 0.06325242414884587


In [ ]:
pred1 = grid_lgb.predict(test_df)

In [ ]:
pred1

array([0.70408294, 0.57900467, 0.42723064, 0.61204889, 0.58646904,
       0.49361015, 0.59747276, 0.57155302, 0.66299768, 0.66065201,
       0.36220007, 0.6747186 , 0.82451745, 0.58228671, 0.52113167,
       0.3057024 , 0.62401759, 0.82661783, 0.55310171, 0.47965651,
       0.24235443, 0.82263244, 0.64299464, 0.62137095, 0.3731414 ,
       0.63481083, 0.85594531, 0.61689853, 0.55473361, 0.39956626,
       0.65857029, 0.81697743, 0.60601616, 0.56245979, 0.32507999,
       0.60039085, 0.77308847, 0.51464062, 0.49180325, 0.26553593,
       0.6242352 , 0.77986304, 0.55216017, 0.50996606, 0.29936955,
       0.60948172, 0.77755519, 0.54446818, 0.49707549, 0.26983248])

In [ ]:
#'석식계'

X_train, X_valid, y_train, y_valid = train_test_split(train_df[train_cols], train_df[target_col2], test_size=0.2, shuffle=True, random_state=34)

lgb_params = {
     'max_depth' : [3,5,7] , 
     'n_estimators': [1000,1200,1500], 
     'learning_rate':[0.01,0.1], 
     'gamma': [0.5], 
     'random_state':[99] }

model = lgb.LGBMRegressor(eval_set=[(X_valid, y_valid)],eval_metric = 'mae')


grid_lgb = GridSearchCV(model, param_grid=lgb_params, cv=3)
grid_lgb.fit(X_train, y_train)
print(grid_lgb.best_params_)

#Get score
print("="*20)
print("Model2 Score: ",grid_lgb.score(X_valid,y_valid))
print("MAE:",mean_absolute_error(y_valid, grid_lgb.predict(X_valid)))

{'gamma': 0.5, 'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 1200, 'random_state': 99}
Model2 Score:  0.757219954262221
MAE: 0.05286469898592169


In [ ]:
pred2 = grid_lgb.predict(test_df)

In [ ]:
pred2

array([0.37991666, 0.29885071, 0.29788347, 0.57869454, 0.44989639,
       0.2261214 , 0.52827201, 0.42453541, 0.52542733, 0.49119274,
       0.22279861, 0.76813417, 0.72785441, 0.41228007, 0.4591476 ,
       0.38138928, 0.73908023, 0.7515066 , 0.4432717 , 0.55337547,
       0.29586291, 0.6894076 , 0.34848144, 0.60009779, 0.4239164 ,
       0.71857479, 0.70812853, 0.42144243, 0.36017495, 0.35750841,
       0.75394077, 0.5962208 , 0.38689232, 0.48952493, 0.26565497,
       0.59935882, 0.60899226, 0.37919902, 0.48427569, 0.31425474,
       0.63820916, 0.61002532, 0.41300346, 0.52928684, 0.23356366,
       0.5838723 , 0.60726045, 0.43402302, 0.36873484, 0.34185915])

In [ ]:
#pred1 = 중식 , pred2 = 석식 두개를 
test_pred = pd.DataFrame([pred1, pred2]).T
test_pred.columns = ['중식계', '석식계']


test_pred = mm.inverse_transform(test_pred)
test_pred = pd.DataFrame(test_pred, columns = ['중식계','석식계'])
test_pred

,중식계,석식계
0,1114.848462,343.824573
1,969.382435,270.459888
2,792.869232,269.584543
3,1007.812856,523.718554
4,978.063490,407.156236
5,870.068608,204.639869
6,990.860825,478.086169
7,960.716160,384.204544
8,1067.066299,475.511730
9,1064.338288,444.529430


In [ ]:
sub = sub.drop(['중식계','석식계'], axis = 1)
sub = pd.concat( [sub,test_pred], axis= 1)
sub

,일자,중식계,석식계
0,2021-01-27,1114.848462,343.824573
1,2021-01-28,969.382435,270.459888
2,2021-01-29,792.869232,269.584543
3,2021-02-01,1007.812856,523.718554
4,2021-02-02,978.063490,407.156236
5,2021-02-03,870.068608,204.639869
6,2021-02-04,990.860825,478.086169
7,2021-02-05,960.716160,384.204544
8,2021-02-08,1067.066299,475.511730
9,2021-02-09,1064.338288,444.529430


In [ ]:
sub.to_csv('sample_submission.csv', encoding = 'utf-8', index = False)

###**LinearReg**

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
# '중식계'

X_train, X_valid, y_train, y_valid = train_test_split(train_df[train_cols], train_df[target_col1], test_size=0.2, shuffle=True, random_state=34)

lin_reg = LinearRegression()

lin_reg.fit(X_train,y_train)
y_valid_predict = lin_reg.predict(X_valid)

print("Model3 Score: ",lin_reg.score(X_valid,y_valid))
print("MAE:",mean_absolute_error(y_valid, lin_reg.predict(X_valid)))

In [ ]:
pred3 = lin_reg.predict(test_df)

In [ ]:
pred3

In [ ]:
# '석식계'

X_train, X_valid, y_train, y_valid = train_test_split(train_df[train_cols], train_df[target_col2], test_size=0.2, shuffle=True, random_state=34)

lin_reg = LinearRegression()

lin_reg.fit(X_train,y_train)
y_valid_predict = lin_reg.predict(X_valid)


print("Model4 Score: ",lin_reg.score(X_valid,y_valid))
print("MAE:",mean_absolute_error(y_valid, lin_reg.predict(X_valid)))

In [ ]:
pred4 = lin_reg.predict(test_df)

In [ ]:
pred4